In [3]:
import dataclasses
import unittest
from src import models, engine, reporting, strategies, data_loader
from datetime import datetime
from src.models import OrderError

assert models.MarketDataPoint
assert engine.Engine
assert reporting.Report
assert strategies.Strategy
assert data_loader.load_market_data


class TestPortfolio(unittest.TestCase):

    def test_open_long_position(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order = models.Order("Buy", "AAPL", 10, 100, "filled")
        portfolio.add_order(order)
        self.assertEqual(portfolio.positions, {"AAPL": "10 @ 100", "Cash": "99000.00"})

    def test_open_short_position(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order = models.Order("Sell", "TSLA", 5, 200, "filled")
        portfolio.add_order(order)
        self.assertEqual(portfolio.positions, {"TSLA": "-5 @ 200", "Cash": "101000.00"})

    def test_close_long_position(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order1 = models.Order("Buy", "AAPL", 10, 100, "filled")
        portfolio.add_order(order1)
        order2 = models.Order("Sell", "AAPL", 10, 150, "filled")
        portfolio.add_order(order2)
        self.assertEqual(portfolio.positions, {"AAPL": "0 @ 0", "Cash": "100500.00"})

    def test_close_short_position(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order1 = models.Order("Sell", "TSLA", 5, 200, "filled")
        portfolio.add_order(order1)
        order2 = models.Order("Buy", "TSLA", 5, 150, "filled")
        portfolio.add_order(order2)
        self.assertEqual(portfolio.positions, {"TSLA": "0 @ 0", "Cash": "100250.00"})

    def test_long_to_short(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order1 = models.Order("Buy", "AAPL", 10, 100, "filled")
        portfolio.add_order(order1)
        order2 = models.Order("Sell", "AAPL", 15, 150, "filled")
        portfolio.add_order(order2)
        self.assertEqual(portfolio.positions, {"AAPL": "-5 @ 150", "Cash": "101250.00"})

    def test_short_to_long(self):
        portfolio = models.Portfolio(initial_cash=100000)
        order1 = models.Order("Sell", "TSLA", 5, 200, "filled")
        portfolio.add_order(order1)
        order2 = models.Order("Buy", "TSLA", 10, 150, "filled")
        portfolio.add_order(order2)
        self.assertEqual(portfolio.positions, {"TSLA": "5 @ 150", "Cash": "99500.00"})


class TestReport(unittest.TestCase):

    def test_execute_trade(self):
        report = reporting.Report()
        Order1 = models.Order("Buy", "AAPL", 10, 100, "filled")
        Portfoilo1 = models.Portfolio(initial_cash=100000)
        Portfoilo1.add_order(Order1)
        report.execute_trade(Order1, log=None, portfolio=Portfoilo1)
        self.assertEqual(
            report.logs, ["BUY 10 of AAPL at price 100. Order status is filled"]
        )
        self.assertEqual(
            report.portfolio_history, [{"AAPL": "10 @ 100", "Cash": "99000.00"}]
        )

    def test_add_successful_trades_to_logs(self):
        report = reporting.Report()
        report.add_trade_log(models.Order("Buy", "AAPL", 10, 100, "filled"))
        report.add_trade_log(models.Order("Sell", "AAPL", 10, 150, "filled"))
        report.add_trade_log(models.Order("Sell", "TSLA", 5, 200, "filled"))
        report.add_trade_log(models.Order("Buy", "TSLA", 5, 150, "filled"))
        self.assertEqual(
            report.logs,
            [
                "BUY 10 of AAPL at price 100. Order status is filled",
                "SELL 10 of AAPL at price 150. Order status is filled",
                "SELL 5 of TSLA at price 200. Order status is filled",
                "BUY 5 of TSLA at price 150. Order status is filled",
            ],
        )

    def test_add_unsuccessful_trades_to_logs(self):
        report = reporting.Report()
        report.add_trade_log(None, "Order was unable to be filled")
        self.assertEqual(report.logs, ["Order was unable to be filled"])


class TestMarketDataPoint(unittest.TestCase):
    def test_MarketDataPoint_creation(self):
        mdp = models.MarketDataPoint(
            timestamp="2023-10-01 10:00:00", symbol="AAPL", price=150.0
        )
        self.assertEqual(mdp.timestamp, "2023-10-01 10:00:00")
        self.assertEqual(mdp.symbol, "AAPL")
        self.assertEqual(mdp.price, 150.0)

    def test_MarketDataPoint_change_price(self):
        with self.assertRaises(dataclasses.FrozenInstanceError):
            mdp = models.MarketDataPoint(
                timestamp="2023-10-01 10:00:00", symbol="AAPL", price=150.0
            )
            mdp.price = 155.0
            self.assertEqual(mdp.price, 155.0)


class TestOrder(unittest.TestCase):
    def test_Order_creation(self):
        order = models.Order(
            action="BUY", symbol="AAPL", quantity=10, price=100, status="filled"
        )
        self.assertEqual(order.action, "BUY")
        self.assertEqual(order.symbol, "AAPL")
        self.assertEqual(order.quantity, 10)
        self.assertEqual(order.price, 100)
        self.assertEqual(order.status, "filled")

    def test_Order_change_status(self):
        order = models.Order(
            action="sell", symbol="GOOGL", quantity=5, price=1500, status="filled"
        )
        order.status = "open"
        self.assertEqual(order.status, "open")

    def test_Order_change_price(self):
        order = models.Order(
            action="sell", symbol="GOOGL", quantity=5, price=1500, status="filled"
        )
        order.price = 1550
        self.assertEqual(order.price, 1550)

    def test_Order_change_quantity(self):
        order = models.Order(
            action="sell", symbol="GOOGL", quantity=5, price=1500, status="filled"
        )
        order.quantity = 10
        self.assertEqual(order.quantity, 10)

    def test_Order_change_action(self):
        order = models.Order(
            action="sell", symbol="GOOGL", quantity=5, price=1500, status="filled"
        )
        order.action = "BUY"
        self.assertEqual(order.action, "BUY")

    def test_Order_change_symbol(self):
        order = models.Order(
            action="sell", symbol="GOOGL", quantity=5, price=1500, status="filled"
        )
        order.symbol = "MSFT"
        self.assertEqual(order.symbol, "MSFT")

    def test_Order_error_status(self):
        with self.assertRaises(OrderError):
            order = models.Order(
                action="buy", symbol="AAPL", quantity=10, price=100, status="filled"
            )
            order.status = "Unfilled"
    
    def test_Order_error_price(self):
        with self.assertRaises(OrderError):
            order = models.Order(
                action="buy", symbol="AAPL", quantity=10, price=50, status="filled"
            )
            order.price = -10
    
    def test_Order_error_quantity(self):
        with self.assertRaises(OrderError):
            order = models.Order(
                action="buy", symbol="AAPL", quantity=1, price=100, status="filled"

            )
            order.quantity = 0
    
        
class TestDataLoader(unittest.TestCase):
    def test_load_market_data(self):
        market_data = data_loader.load_market_data("data/test_data.csv")
        
        mdp1 = models.MarketDataPoint(
                timestamp= datetime.fromisoformat("2025-09-20T19:40:35.392907"), symbol="AAPL", price=145.99
            )
        mdp2 = models.MarketDataPoint(
                timestamp= datetime.fromisoformat("2025-09-20T19:40:35.403503"), symbol="AAPL", price=146.98
            )
        mdp3 = models.MarketDataPoint(
                timestamp= datetime.fromisoformat("2025-09-20T19:40:35.416050"), symbol="AAPL", price=147.32
           )
        self.assertEqual(market_data[0], mdp1)
        self.assertEqual(market_data[1], mdp2)
        self.assertEqual(market_data[2], mdp3)
        self.assertEqual(len(market_data), 3)
       
        


if __name__ == "__main__":
    unittest.main(argv=["ignored", "-v"], exit=False)

test_load_market_data (__main__.TestDataLoader.test_load_market_data) ... ok
test_MarketDataPoint_change_price (__main__.TestMarketDataPoint.test_MarketDataPoint_change_price) ... ok
test_MarketDataPoint_creation (__main__.TestMarketDataPoint.test_MarketDataPoint_creation) ... ok
test_Order_change_action (__main__.TestOrder.test_Order_change_action) ... ok
test_Order_change_price (__main__.TestOrder.test_Order_change_price) ... ok
test_Order_change_quantity (__main__.TestOrder.test_Order_change_quantity) ... ok
test_Order_change_status (__main__.TestOrder.test_Order_change_status) ... ok
test_Order_change_symbol (__main__.TestOrder.test_Order_change_symbol) ... ok
test_Order_creation (__main__.TestOrder.test_Order_creation) ... ok
test_Order_error_price (__main__.TestOrder.test_Order_error_price) ... ok
test_Order_error_quantity (__main__.TestOrder.test_Order_error_quantity) ... ok
test_Order_error_status (__main__.TestOrder.test_Order_error_status) ... ok
test_close_long_position (__m